<a href="https://colab.research.google.com/github/GBakalkinOAI/DDLS2024/blob/main/CellxGene_Census_scVI_Monocytes_06_with_API_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is my second attempt, where I scraped CELLxGENE Census github page for .md, .py and .ipynb files, combined them and attached to chatGPT https://chatgpt.com/share/670904ff-a098-8001-b522-56d7bb2bf25a

In [ ]:
# Installation commands are taken from: https://docs.scvi-tools.org/en/latest/tutorials/notebooks/hub/cellxgene_census_model.html
# See also: https://chanzuckerberg.github.io/cellxgene-census/cellxgene_census_docsite_installation.html
!pip install --quiet scvi-colab
!pip install --quiet cellxgene-census
!pip install --quiet pybiomart
from scvi_colab import install

install()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.9.0 which is incompatible.
spatialdata 0.2.3 requires fsspec<=2023.6, but you have fsspec 2024.9.0 which is incompatible.
INFO     scvi-colab: Installing scvi-tools.                                                                        


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cellxgene-census 1.16.2 requires s3fs>=2021.06.1, but you have s3fs 0.6.0 which is incompatible.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2023.6.0 which is incompatible.


INFO     scvi-colab: Install successful. Testing import.                                                           


In [ ]:
# Import necessary libraries
import cellxgene_census
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scvi

In [ ]:
# Select the CELLxGENE Census snapshot, see detals of dataset's scheme here:
#   https://raw.githubusercontent.com/chanzuckerberg/cellxgene-census/refs/heads/main/docs/cellxgene_census_schema.md
#   https://raw.githubusercontent.com/chanzuckerberg/single-cell-curation/refs/heads/main/schema/5.0.0/schema.md
dataset_version = "2024-07-01"

# Define what do we extract from Census
emb_names = ["scvi"]  # specify the embedding you are interested in
organism = "homo_sapiens"
cell_type_counts_min = 5 # drop cells if the corresponding cell type has too few cells in a particular donor
cell_type_donors_min = 5 # drop cells if too few donors have the corresponding cell type

In [ ]:
# Experiment: get all features/genes, see https://chanzuckerberg.github.io/cellxgene-census/notebooks/analysis_demo/comp_bio_explore_and_load_lung_data.html
#with cellxgene_census.open_soma(census_version=dataset_version) as census:
#  CxG_var = cellxgene_census.get_var(census, organism)

pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 120)
#CxG_var

In [ ]:
# Experiment: get all development_stage values
with cellxgene_census.open_soma(census_version=dataset_version) as census:
    cell_metadata = cellxgene_census.get_obs(
        census,
        organism,
        value_filter = "sex == 'male' and is_primary_data == True",
        column_names = [
            'development_stage_ontology_term_id', 'development_stage'
            ]
    )

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

age_filter = r'year-old|decade|adult|middle aged' # older than 1 year old
age_filter_relaxed = r'year-old|decade|adult|middle aged|'
# unknown|under-1-year-old|adolescent|young adult|adult|early adulthood|late adulthood|middle aged|25-44 year-old|80 year-old and over|human aged|

# Double-check the removed stages
# cell_metadata[~cell_metadata['development_stage'].str.contains(age_filter, case=False, na=False)]['development_stage'].unique().tolist()
cell_metadata[ cell_metadata['development_stage'].str.contains(age_filter, case=False, na=False)]['development_stage'].unique().tolist()


# cell_metadata = cell_metadata[cell_metadata['development_stage'].str.contains(age_filter, case=False, na=False)]
# print( cell_metadata.value_counts(subset=['development_stage_ontology_term_id', 'development_stage']) )

['26-year-old human stage',
 '41-year-old human stage',
 'human adult stage',
 'human late adulthood stage',
 'human early adulthood stage',
 '78-year-old human stage',
 '60-year-old human stage',
 '64-year-old human stage',
 '69-year-old human stage',
 '65-year-old human stage',
 '74-year-old human stage',
 '55-year-old human stage',
 '32-year-old human stage',
 '45-year-old human stage',
 '61-year-old human stage',
 '66-year-old human stage',
 '51-year-old human stage',
 '49-year-old human stage',
 '5-year-old human stage',
 '54-year-old human stage',
 '21-year-old human stage',
 '22-year-old human stage',
 '6-year-old human stage',
 '29-year-old human stage',
 '23-year-old human stage',
 '59-year-old human stage',
 '35-year-old human stage',
 '37-year-old human stage',
 '82-year-old human stage',
 '57-year-old human stage',
 '44-year-old human stage',
 '79-year-old human stage',
 '42-year-old human stage',
 '71-year-old human stage',
 '86-year-old human stage',
 '68-year-old human s

In [ ]:
with cellxgene_census.open_soma(census_version=dataset_version) as census:
  # Which version of schema do we use exactly?
  census_summary = census["census_info"]["summary"].read().concat().to_pandas()

  # To add citations and human readable names to `dataset_id` we can augment
  # .get_obs() results with .merge(census_datasets, on="dataset_id")
  census_datasets = (
      census["census_info"]["datasets"]
      .read(column_names=[
          # "citation", # will need it for the publication
          # "collection_name", "collection_doi",
          "dataset_title", # human readable name
          # "dataset_h5ad_path", # downloable from AWS using these file names
          "dataset_total_cell_count", # how many cells are contributed (including duplicated cells)
          "dataset_id" # key for .get_obs() results
          ])
      .concat().to_pandas()
      .set_index("dataset_id")
  )

  # Census summary with counts per cell type
  # `label` is specific for each `category`, allowing further zoom-in
  # number of labels per each `category` is as follows:
  # 1 all  (label is 'na')
  # 1 suspension_type (label is 'cell')
  # 3 sex (label is 53.7% 'male', 41.2% 'female' and 5.1% 'unknown')
  # 24 assay
  # 31 self_reported_ethnicity
  # 55 tissue_general
  # 109 disease
  # 267 tissue
  # 698 cell_type
  census_summary_cell_counts = (
      census["census_info"]["summary_cell_counts"]
      .read().concat().to_pandas()
      .query("organism == 'Homo sapiens'") # Not like in `organism`, different spelling
  )

print( census_summary )
# General information for all schemes: https://raw.githubusercontent.com/chanzuckerberg/cellxgene-census/refs/heads/main/docs/cellxgene_census_schema.md
# Specific details for our schema v.5.0.0: https://raw.githubusercontent.com/chanzuckerberg/single-cell-curation/refs/heads/main/schema/5.0.0/schema.md

# census_datasets # has 812 datasets, nothing interesting to see yet

   soma_joinid                       label       value
0            0       census_schema_version       2.0.1
1            1           census_build_date  2024-05-20
2            2      dataset_schema_version       5.0.0
3            3            total_cell_count   115556140
4            4           unique_cell_count    60597966
5            5  number_donors_homo_sapiens       17651
6            6  number_donors_mus_musculus        4216


In [ ]:
# Search all cell types containing pattern "*monocyte*" so capture all cell types related to Monocytes
cell_types_monocyte = census_summary_cell_counts.query("category == 'cell_type' and label.str.contains('monocyte', case=False, na=False)")
cell_types_monocyte[['label', 'ontology_term_id', 'unique_cell_count']]

,label,ontology_term_id,unique_cell_count
158,granulocyte monocyte progenitor cell,CL:0000557,5893
160,promonocyte,CL:0000559,10669
165,monocyte,CL:0000576,314205
262,classical monocyte,CL:0000860,1059249
265,non-classical monocyte,CL:0000875,178362
322,CD14-positive monocyte,CL:0001054,580910
350,"CD14-positive, CD16-negative classical monocyte",CL:0002057,261782
433,intermediate monocyte,CL:0002393,6148
435,"CD14-low, CD16-positive monocyte",CL:0002396,87379
436,"CD14-positive, CD16-positive monocyte",CL:0002397,20045


In [ ]:
# Lets examine male cells metadata (39.9M cells x 26 columns) and decide which columns do we really need.
# In the command we name every possible metadata column, and then
# - label it #drop# and comment out if we do not need it
# - label it #ToDo# if we need to investigate it further
# - label it #batch#ToDo# if this column may cause batch effects, ToDo: check if it cause significant batch effect
# - label it #batch# if we needed this column to stratify batch effect or it is confounded with LOY
monocyte_filter = " or ".join(["cell_type == '{}'".format(label) for label in (cell_types_monocyte['label'].tolist())])
monocyte_filter = (
    "sex == 'male' and is_primary_data == True and (" +
    monocyte_filter + ")"
)

with cellxgene_census.open_soma(census_version=dataset_version) as census:
    cell_metadata = cellxgene_census.get_obs(
        census,
        organism,
        value_filter = monocyte_filter,
        column_names = [
            'soma_joinid', # after filtering we will use it to download the right cells
            # 'sex', 'sex_ontology_term_id', # filter "male" to study LOY
            # 'is_primary_data', # 56% True, filter True to look only at non-duplicate data
            'dataset_id', #batch#ToDo# 11.3% + 4% + 3.6% + 3.1% + 2.8% + ...
            # 'assay', 'assay_ontology_term_id', 'suspension_type', #batch#ToDo# single cell/nuclei technology
            'cell_type_ontology_term_id', 'cell_type', #batch# later we will filter for LOY-enriched cell types
            'development_stage_ontology_term_id', 'development_stage', #batch# age, filter >=20 years old, 7% '50-year-old human stage'
            'disease_ontology_term_id', 'disease', #batch# 70% healthy + 11% covid
            # 'observation_joinid', # unique observation identifier for each cell
            'self_reported_ethnicity_ontology_term_id', # 'self_reported_ethnicity',  #batch#ToDo# 52% 'unknown', 40% 'European'
            'tissue_ontology_term_id', 'tissue', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id', #batch#ToDo# cell types will filter tissues automatically?
            'raw_sum', #ToDo# Is this like Seurat's nReads?
            'nnz', 'raw_mean_nnz', 'raw_variance_nnz', # Is nnz like Seurat's nUMIs? Mean/variance over what?
            'n_measured_vars', # Is this like Seurat's nFeatures ?
            'donor_id' #batch# We study LOY withing each donor, then look at inter-donor variability of DEGs
            ]
    )

In [ ]:
# cell_metadata_old = cell_metadata.copy()
cell_metadata = cell_metadata_old.copy()

In [ ]:
# Only warn if missing data is actually present, print columns with missing data, and remove missing values
missing_data = cell_metadata.isna().sum()
missing_columns = missing_data[missing_data > 0]
if not missing_columns.empty:
    print("Warning: Missing data found in the following columns:")
    print(missing_columns)
cell_metadata = cell_metadata.dropna(subset=['cell_type', 'donor_id', 'development_stage', 'disease'])

In [ ]:
# Subsample 10000 random rows to avoid crashing Colab's RAM
cell_metadata = cell_metadata.sample(n=10000, random_state=1)

In [ ]:
# Because we have some follow up experiments, same 'donor_id' can have different 'disease' and/or 'development_stage'
cell_metadata['donor_follow_up'] = (
    cell_metadata['donor_id'].astype(str) + '_' +
    cell_metadata['disease'].astype(str) + '_' +
    cell_metadata['development_stage'].astype(str)
)

#cell_metadata['n_disease_per_donor'] = cell_metadata.groupby(['donor_id'], observed=False)['disease'].transform('nunique')
#cell_metadata['n_development_stage_per_donor'] = cell_metadata.groupby(['donor_id'], observed=False)['development_stage'].transform('nunique')
#filtered_data = cell_metadata[(cell_metadata['n_disease_per_donor'] > 1) | (cell_metadata['n_development_stage_per_donor'] > 1)]
#filtered_data = filtered_data.sort_values(by=['donor_id', 'disease', 'development_stage'])
#print(filtered_data[['donor_id', 'development_stage', 'disease']].value_counts(sort=False))

In [ ]:
# First, drop cells if the corresponding cell type has too few cells in the corresponding donor_follow_up
cell_metadata['cell_type_size'] = cell_metadata.groupby(['cell_type', 'donor_follow_up'], observed=False)['cell_type'].transform('size')
cell_metadata = cell_metadata[cell_metadata['cell_type_size'] >= cell_type_counts_min]
# cell_metadata[['cell_type', 'donor_follow_up', 'cell_type_size']].sort_values(by=['cell_type', 'donor_follow_up'])

In [ ]:
# Next, drop cells if too few donors still have the corresponding cell type
cell_metadata['cell_type_donors'] = cell_metadata.groupby('cell_type', observed=False)['donor_follow_up'].transform('nunique')
cell_metadata = cell_metadata[cell_metadata['cell_type_donors'] >= cell_type_donors_min]
# cell_metadata[['cell_type', 'donor_follow_up', 'cell_type_size', 'cell_type_donors']].sort_values(by=['cell_type', 'donor_follow_up'])

In [ ]:
print( cell_metadata.value_counts(subset=['cell_type', 'donor_id', 'development_stage', 'disease']).sort_values() )

cell_type                                        donor_id            development_stage         disease                    
non-classical monocyte                           pt12                61-year-old human stage   B-cell non-Hodgkin lymphoma       5
CD14-positive monocyte                           SG_HEL_H195         64-year-old human stage   normal                            5
CD14-positive, CD16-negative classical monocyte  P-HC023             30-year-old human stage   normal                            5
monocyte                                         S4ECX8ng            46-year-old human stage   normal                            5
CD14-low, CD16-positive monocyte                 LONZA3038306        30-year-old human stage   normal                            5
                                                                                                                              ... 
CD14-positive, CD16-negative classical monocyte  P-S085              62-year-old human stag

In [ ]:
# print(cell_metadata.value_counts(subset=['dataset_id'], normalize=True).head(3)) #ToDo# check batch effects of contributing datasets

In [ ]:
print(cell_metadata.value_counts(subset=['cell_type_ontology_term_id', 'cell_type'], normalize=True).head(10)) # most abundant cell types

In [ ]:
print(cell_metadata.value_counts(subset=['tissue_ontology_term_id', 'tissue', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id'], normalize=True).head(10))

In [ ]:
obs_coords_ids = cell_metadata["soma_joinid"].tolist()
with cellxgene_census.open_soma(census_version=dataset_version) as census:
    adata = cellxgene_census.get_anndata(
        census,
        organism=organism,
        measurement_name="RNA",
        obs_coords=obs_coords_ids,
        obs_embeddings=emb_names
    )

adata

AnnData object with n_obs × n_vars = 8026 × 60530
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'observation_joinid', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'
    obsm: 'scvi'

In [ ]:
census.close()
del census